# atlas divied

In [ ]:
#為了後續特徵提取方便，把170個腦區圖譜分開存為單獨紀錄一個腦區的nii.gz檔，小於10voxel的腦區會列出來並於特徵提取時忽略。
label_name = './data_final/MNI/AAL3_MNI.nii.gz'
output_name = './data_final/MNI/AAL3/AAL3_'
label_nii = nib.load(label_name)
label_img = np.asarray(label_nii.dataobj)
print(label_img.shape)
print(label_img.max())
for i in range(1,int(label_img.max())+1):
    modified_img = label_img.copy()
    modified_img[modified_img!=i] = 0
    modified_img[modified_img==i] = 1
    if(sum(sum(sum(modified_img))))<10:
        print(i,"has only",sum(sum(sum(modified_img))),"voxel")
        
    modified_img = nib.Nifti1Image(modified_img, label_nii.affine, label_nii.header)
    nib.save(modified_img, output_name+'{:03d}'.format(i)+".nii.gz")

# radiomics feature extration(fusion)

In [ ]:
import radiomics
from radiomics import featureextractor
import os
import nibabel as nib
import numpy as np
import pandas as pd

In [ ]:
import logging
# close waring from radiomics(it will imporve speed)
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)

In [ ]:
file_name = "../data_final/origin data/"
datalist = os.listdir(file_name)
skip = [157]
len(datalist)

In [ ]:
lable_file_name = "../data_final/MNI/AAL3/"
lable_datalist = os.listdir(lable_file_name)
#35,36,81,82腦區被分割成更細的子腦區因此在我們的AAL3圖譜內voxel為0，133,134,160,167,168,170則是voxel太小因此忽略計算。
lable_skip=[35,36,81,82,133,134,160,167,168,170]
len(lable_datalist)

In [ ]:
extractor = featureextractor.RadiomicsFeatureExtractor(correctMask=True)
extractor.enableImageTypeByName(imageType="Wavelet")
extractor.enableImageTypeByName(imageType="LoG")
extractor.enableImageTypeByName(imageType="Square")
extractor.enableImageTypeByName(imageType="SquareRoot")
extractor.enableImageTypeByName(imageType="Logarithm")
extractor.enableImageTypeByName(imageType="Exponential")
extractor.enableImageTypeByName(imageType="Gradient")
extractor.enableImageTypeByName(imageType="LBP2D")

In [ ]:
#pyradiomics跑出來的結果中，這幾個欄位是紀錄文字資訊(如版本號)，因此刪除。
radiomics_del_ls = [0,1,2,3,4,5,6,7,8,9,10,14,15,16,17,20,21]
imgname = []
df = pd.DataFrame()
for i in range(len(datalist)):
    print("now dealing i=",i,", name is",datalist[i])
    DWI_name = file_name+datalist[i]+'/diff2MNI.nii.gz'
    

    DWI_nii = nib.load(DWI_name)
    DWI_img = np.asarray(DWI_nii.dataobj)
    
    if(i not in skip):
        imgname.append(datalist[i])
        temp = pd.DataFrame()
        for j in range(len(lable_datalist)):
            if j+1 not in lable_skip:
                label_name = lable_file_name+lable_datalist[j]
                #print(label_name)
                features = extractor.execute(DWI_name,label_name)
                df_new = pd.DataFrame.from_dict(features.values()).T
                df_new.columns = features.keys()
                df_new = df_new.drop(df_new.columns[radiomics_del_ls],axis = 1)
                df_new = df_new.rename(columns=lambda x: x +'_'+ '{:03d}'.format(j+1))
                temp = pd.concat([temp,df_new],axis=1)

        df = pd.concat([df,temp])
    else:
        print(datalist[i]," is skiped.")
df.insert(0, "ID", imgname)
df

In [ ]:
#save features to csv file
df.to_csv("data_final/0416_fuse_radiomics_output.csv",index=False)